In [1]:
# libary imports 
import os
import torch
from torch import nn
import torchvision
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
from torchvision import transforms
import torchvision.datasets as datasets

from sklearn import metrics
from sklearn import decomposition
from sklearn import manifold
from tqdm.notebook import trange, tqdm

import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data

import matplotlib.pyplot as plt
import numpy as np
import copy
import random
import time


from functools import reduce
import operator
import torch.utils.data as data_utils
from sklearn.metrics import classification_report


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [58]:
from torch.utils.data import Subset


# Prepare CIFAR-10 dataset
trainset = CIFAR10(os.getcwd(), download=True, transform=transforms.Compose(
      [transforms.ToTensor(),
      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]))

testset = CIFAR10(os.getcwd(), download=True, transform=transforms.Compose(
      [transforms.ToTensor(),
      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]))


#print(images, labels)

def my_fun(dataset):
    X=[]
    y=[]
    for feature, label in iter(dataset):
        X.append(feature)
        y.append(label)
    return X, y

   
X_train, y_train = my_fun(trainset)
X_test, y_test =my_fun(testset)


Files already downloaded and verified
Files already downloaded and verified


In [115]:

class MLPClassifer(nn.Module):
      '''
        Multilayer Perceptron.
      '''
      def __init__(self ):
        super().__init__()
        input=trainset.data[1].shape
        output=len(torch.unique(torch.tensor(trainset.targets)))
        hidden_layer_sizes=(64,32)
        self.layers = nn.Sequential(
          nn.Flatten(),
          nn.Linear(reduce(operator.mul, input),  hidden_layer_sizes[0]),
          nn.ReLU(),
          nn.Linear(hidden_layer_sizes[0],  hidden_layer_sizes[1]),
          nn.ReLU(),
          nn.Linear(hidden_layer_sizes[1], output)
        )

      def forward(self, x):
        '''Forward pass'''
        return self.layers(x)

class Neural_Network(object):
    
    
    def __init__ (self,batch_size=2000,epochs=2,verbose=True,loss_function = nn.CrossEntropyLoss(),
         optimiser='adam',classifier= MLPClassifer(), 
         learning_rate=1e-3,  betas=(0.9,0.99) , eps=1e-8, weight_decay=0, amsgrad=False,
      momentum=0.9, nesterov=True, dampening=0, maximise=False):
      
      self.batch_size=batch_size
      self.epochs=epochs
      
      self.verbose=True
      #used inside the training/ testing loops
      self.running_loss = 0
      self.loss_function = loss_function
      self.classifier= classifier
      
      self.sm = torch.nn.Softmax()

      #some initial parameters for either standard optimisers
      self.learning_rate=learning_rate 
      self.betas= betas
      self.eps= eps
      self.weight_decay=weight_decay
      self.amsgrad=amsgrad 
      self.momentum=momentum
      self.nesterov=nesterov
      self.dampening=dampening
      self.maximise=maximise

      #default optimiser: adam 
      self.optimiser=self.set_optimiser_2(optimiser)
    
    #for some reason calling this function set_optimiser returns None 
    def set_optimiser_2(self, optimiser):
        #playing around with adam and sgd only at the moment
        #there is only two of them 
        if optimiser.lower().strip()=='adam':
            self.optimiser=torch.optim.Adam(self.classifier.parameters(),lr=self.learning_rate, betas=self.betas, eps=self.eps, weight_decay=self.weight_decay, amsgrad=self.amsgrad)
        elif optimiser.lower().strip()=='sgd':
            self.optimiser=torch.optim.SGD(self.classifier.parameters(),lr=self.learning_rate, weight_decay=self.weight_decay, momentum=0.9,nesterov=self.nesterov , dampening=self.dampening)
        return self.optimiser


    def get_optimiser(self):
      print(self.optimiser)

    
    def get_params(self): #get parameters
       if (self.optimiser != None) :
         return  (self.props(),self.optimiser.state_dict(), self.classifier.state_dict)
       else: return self.classifier.state_dict() 
  
    def fit(self,X,y):
        train = [*zip(X,y)]
        trainloader =  torch.utils.data.DataLoader(train,batch_size=8, shuffle=False, num_workers=2)
        for epoch in range(self.epochs):  # loop over the dataset 2 times
        
          running_loss =self.running_loss
          for i, data in enumerate(trainloader, 0):
          # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
             
            # zero the parameter gradients
            self.optimiser.zero_grad()

            # forward + compute loss+ backward + optimize
            outputs = self.classifier(inputs)
            loss = self.loss_function(outputs, labels)
            loss.backward()
            self.optimiser.step()
            # print statistics
            running_loss += loss.item()
            if i % self.batch_size == 1999:    # print every 2000 mini-batches
                if self.verbose==True:
                  print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / self.batch_size:.3f}')
                running_loss = 0.0

    def predict(self,X,y): #Predict using the multi-layer perceptron classifier.
      test = [*zip(X,y)]
      testloader = torch.utils.data.DataLoader(test,batch_size=8, shuffle=False, num_workers=2)
      prediction_list=[]
      # again no gradients needed
      with torch.no_grad():
          for data in testloader:
              images, labels = data
              outputs = self.classifier(images)
              estimation, predictions = torch.max(outputs, 1)
              prediction_list.append(predictions)
      return prediction_list #torch.stack(prediction_list)


    def predict_log_proba(self,X,y):  #	Return the log of probability estimates.
        data = [*zip(X,y)]
        loader =  torch.utils.data.DataLoader(data,batch_size=8, shuffle=False, num_workers=2)
        y_prob=self.predict_proba(loader)
        log_proba=np.log(y_prob)
        return log_proba
 

    def predict_proba(self,X,y):	#Probability estimates.
      probabilities_list=[]
      data = [*zip(X,y)]
      loader =  torch.utils.data.DataLoader(data,batch_size=8, shuffle=False, num_workers=2)
      # again no gradients needed
      with torch.no_grad():
          for data in loader:
              images, labels = data
              outputs = self.classifier(images)
              probabilities_list.append(self.sm(outputs) )
              estimation, predictions = torch.max(outputs, 1)
      return probabilities_list
      

    def score(self,X,y): #Return the mean accuracy on the given test data and labels.
        prediction_list=[]
        targets_list=[]
        data = [*zip(X,y)]
        loader =  torch.utils.data.DataLoader(data,batch_size=8, shuffle=False, num_workers=2)
        num_classes=len(torch.unique(torch.tensor(y)))
        #set it to true if you have a list of available labels in text format
        labels_available=False
        if labels_available:
            classes = ('plane', 'car', 'bird', 'cat',
                    'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
            # prepare to count predictions for each class
            correct_pred = {classname: 0 for classname in classes}
            total_pred = {classname: 0 for classname in classes}
        #use the class number as key
        else: 
            classes=tuple(np.arange(num_classes))  
            correct_pred = {classname: 0 for classname in np.arange(num_classes)}
            total_pred= {classname: 0 for classname in np.arange(num_classes)}

        # again no gradients needed
        with torch.no_grad():
            for data in loader:
                images, labels = data
                targets_list.append(labels)
                outputs = self.classifier(images)
                estimation, predictions = torch.max(outputs, 1)
                prediction_list.append(predictions)
                # collect the correct predictions for each class
                for label, prediction in zip(labels, predictions):
                    if label == prediction:
                        correct_pred[classes[label]] += 1
                    total_pred[classes[label]] += 1
        accuracies={}
        # print accuracy for each class
        for classname, correct_count in correct_pred.items():
            accuracy = 100 * float(correct_count) / total_pred[classname]
            accuracies[classname]=accuracy
            if self.verbose:
              print(f'Accuracy for class: {classname} is {accuracy:.1f} %')
        return targets_list,prediction_list
        

    def set_params(self, attr, value): #sets parameters
        setattr(self, attr, value)
        
    def props(cls):   
        return [i for i in cls.__dict__.items() if i[:1] != '_']

    def get_classifier(self):
        return self.classifier()

    def set_optimiser(self, optimiser_choice):
        return 

    def get_classification_report(y, y_pred):
        y_prime = np.stack([np.stack([d for d in d_]) for d_ in y]).flatten()
        y_prime=np.stack([np.stack([d for d in d_]) for d_ in y_pred]).flatten()   
        print(classification_report(y_prime, y_test))
    


In [116]:
model= Neural_Network()
model.set_optimiser_2('adam')
model.fit(X_train,y_train)


[1,  2000] loss: 1.795
[1,  4000] loss: 1.652
[1,  6000] loss: 1.626
[2,  2000] loss: 1.562
[2,  4000] loss: 1.515
[2,  6000] loss: 1.522


In [118]:
y, y_pred=model.score(X_test, y_test)

Accuracy for class: 0 is 46.0 %
Accuracy for class: 1 is 64.5 %
Accuracy for class: 2 is 22.9 %
Accuracy for class: 3 is 42.5 %
Accuracy for class: 4 is 32.8 %
Accuracy for class: 5 is 30.7 %
Accuracy for class: 6 is 63.3 %
Accuracy for class: 7 is 62.6 %
Accuracy for class: 8 is 70.8 %
Accuracy for class: 9 is 56.3 %


In [149]:
print(classification_report(y_prime, y_pred_prime))

              precision    recall  f1-score   support

           0       0.62      0.46      0.53      5000
           1       0.59      0.65      0.61      5000
           2       0.44      0.23      0.30      5000
           3       0.32      0.42      0.36      5000
           4       0.47      0.33      0.39      5000
           5       0.44      0.31      0.36      5000
           6       0.45      0.63      0.52      5000
           7       0.52      0.63      0.57      5000
           8       0.57      0.71      0.63      5000
           9       0.54      0.56      0.55      5000

    accuracy                           0.49     50000
   macro avg       0.49      0.49      0.48     50000
weighted avg       0.49      0.49      0.48     50000

